In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(".env")

# 手动设置环境变量
# os.environ["OPENAI_API_BASE_URL"] = "https://ai-yyds.com/v1"
# os.environ["OPENAI_API_KEY"] = "sk-XXXXXXXXXXXXXXXXXXXX0bDb"
# os.environ["MODEL_NAME"] = "test"

## 在链上使用记忆
在 LangChain 中，记忆（Memory）是指链（Chain）或代理（Agent）记住先前对话或交互的能力。这对于构建聊天机器人、对话系统和其他需要保持上下文的应用程序至关重要。LangChain 提供了多种记忆工具，允许您在链中使用它们，以实现不同的记忆行为。
+ 为什么在链中使用记忆？
  + 保持对话上下文：
    + 记忆可以帮助链记住之前的对话内容，并在生成响应时考虑这些内容，从而使对话更加自然和连贯。
  + 提高模型的准确性：
    + 记忆可以帮助链更好地理解用户的意图，并生成更加准确的响应。
  + 实现更复杂的对话逻辑：
    + 记忆可以帮助链跟踪对话中的实体、关系和其他信息，从而实现更复杂的对话逻辑。

+ LangChain 中的记忆类型
LangChain 提供了多种记忆类型，每种类型都有不同的特点和适用场景：
  + ConversationBufferMemory：
    + 这是最简单的记忆形式，它将所有对话消息存储在一个缓冲区中。
    + 它保留了完整的对话历史记录。
    + 适用于短对话。
  + ConversationBufferWindowMemory：
    + 这种记忆形式只保留最近的“k”个交互。
    + 它通过限制存储的对话历史记录来解决上下文窗口限制的问题。
    + 适用于中等长度的对话。
  + ConversationSummaryMemory：
    + 这种记忆形式总结了对话的早期部分，只保留最近的交互。
    + 它使用 LLM 来创建对话的摘要，从而减少了需要存储的令牌数量。
    + 适用于长对话。
  + ConversationKGMemory：
    + 这种记忆形式使用知识图谱来存储对话信息。
    + 它提取对话中的实体和关系，并将其存储在知识图谱中。
    + 适用于需要进行知识推理的对话。
  + ConversationEntityMemory：
    + 跟踪对话中提到的实体，并随着对话的进行不断更新这些实体的信息。
    + 使用LLM从对话文本中提取实体。
    + 适用于需要跟踪特定实体的对话。
  + ConversationSummaryBufferMemory：
    + 在内存中保留最近交互的缓冲区，同时将较早的交互编译成摘要。
    + 使用token长度而不是交互次数来确定何时刷新交互。
    + 适用于长对话，并且需要尽量保持对话的完整上下文时。
  + ConversationTokenBufferMemory：
    + 它与 ConversationBufferMemory 类似，但它使用令牌（tokens）的数量而不是消息的数量来确定何时截断对话历史记录。
    + 适用于需要精确控制上下文长度的对话。

### LLMChain 使用记忆

In [ ]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
import os

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

# 自定义模板
template = """你是一个可以和人类对话的机器人.
{chat_history}
人类:{human_input}
机器人:"""

prompt= PromptTemplate(
    template=template,
    input_variables=["chat_history", "human_input"],
)
memory = ConversationBufferMemory(
    memory_key="chat_history",
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [ ]:
chain.predict(human_input="我最新喜欢我的世界这个游戏，你还记得我叫什么吗？")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import  (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import  SystemMessage

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="你好，我是一个可以和人类对话的机器人",
            role="system",
        ),
        MessagesPlaceholder(
            variable_name="chat_history",
        ),
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),
    ]
)

# print(prompt.format(human_input="你好",chat_history=[]))

In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [ ]:
chain.predict(human_input="我叫tomie，我是一个AI应用程序猿")

### ConversationChain

In [ ]:
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
)
chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,
)

In [ ]:
chain.predict(input="帮我做个一日游攻略")

In [ ]:
# 自定义一下，对其进行覆盖
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

template = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'.
当前对话:
{history}
Human:{input}
AI助手:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["history", "input"],
)

chain = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(
        ai_prefix="AI助手",
        return_messages=True,
    ),
    prompt=prompt,
    verbose=True,
)

In [ ]:
chain.predict(input="你知道我叫什么名字吗?")

### 同一个链合并使用多个memory

In [ ]:
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory,
    CombinedMemory
)
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

# 使用ConversationSummaryMemory对对话进行总结
summary = ConversationSummaryMemory(
    llm=llm,
    input_key="input"
)
# 使用ConversationBufferMemory对对话进行缓存
cov_memory = ConversationBufferMemory(
    memory_key="history_now",
    input_key="input",
)

memory = CombinedMemory(
    memories=[summary, cov_memory],
)

TEMPLATE = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'.
之前的对话摘要:
{history}
当前对话:
{history_now}
Human:{input}
AI："""

prompt = PromptTemplate(
    template=TEMPLATE,
    input_variables=["history", "history_now", "input"],
)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [ ]:
chain.run("那ETH呢？")

### 多参数链增加记忆

In [ ]:
from langchain.embeddings.openai import  OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

with open("./sources/letter.txt") as f:
    text = f.read()
#切分文本
text_splitter = CharacterTextSplitter(
    chunk_size = 20,
    chunk_overlap = 5
)
texts = text_splitter.split_text(text)

#使用openai的embedding
embeddings = OpenAIEmbeddings()
#使用chroma向量存储
docssearch = Chroma.from_texts(
    texts,
    embeddings,
)
query = "公司有什么新策略?"
docs = docssearch.similarity_search(query=query)

In [ ]:
#构建问答对话链
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

template = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'，参考一下相关文档以及历史对话信息，AI会据此组织最终回答内容.
{context}
{chat_history}
Human:{human_input}
AI:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "chat_history", "human_input"],
)

#使用ConversationBufferMemory对对话进行缓存 
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="human_input",
    return_messages=True,
)

#加载对话链
chain = load_qa_chain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
    chain_type="stuff"
)

chain({"input_documents":docs, "human_input":"公司的营销策略是什么？"})
